# Iterating on LLM Apps with TruLens

Our simple RAG often struggles with retrieving not enough information from the insurance manual to properly answer the question. The information needed may be just outside the chunk that is identified and retrieved by our app. Reducing the size of the chunk and adding "sentence windows" to our retrieval is an advanced RAG technique that can help with retrieving more targeted, complete context. Here we can try this technique, and test its success with TruLens.

In [1]:
import openai
from trulens_eval import Tru
tru = Tru(database_redact_keys=True)

Package protobuf is installed but has a version conflict:
	(protobuf 3.20.0 (c:\users\28263\anaconda3\lib\site-packages), Requirement.parse('protobuf>=4.23.2'))

This package is optional for trulens_eval so this may not be a problem but if
you need to use the related optional features and find there are errors, you
will need to resolve the conflict:

    ```bash
    pip install 'protobuf>=4.23.2'
    ```

If you are running trulens_eval in a notebook, you may need to restart the
kernel after resolving the conflict. If your distribution is in a bad place
beyond this package, you may need to reinstall trulens_eval so that all of the
dependencies get installed and hopefully corrected:
    
    ```bash
    pip uninstall -y trulens_eval
    pip install trulens_eval
    ```

Package watchdog is installed but has a version conflict:
	(watchdog 2.1.6 (c:\users\28263\anaconda3\lib\site-packages), Requirement.parse('watchdog>=3.0.0'))

This package is optional for trulens_eval so this may not be

🦑 Tru initialized with db url sqlite:///default.sqlite .
🔒 Secret keys will not be included in the database.


## Load data and test set

In [4]:
from llama_index.readers.smart_pdf_loader import SmartPDFLoader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "https://arxiv.org/pdf/1910.13461.pdf"  # also allowed is a file path e.g. /home/downloads/xyz.pdf
pdf_loader = SmartPDFLoader(llmsherpa_api_url=llmsherpa_api_url)
documents = pdf_loader.load_data(pdf_url)
# Load some questions for evaluation
honest_evals = [
    "What are the typical coverage options for homeowners insurance?",
    "What are the requirements for long term care insurance to start?",
    "Can annuity benefits be passed to beneficiaries?",
    "Are credit scores used to set insurance premiums? If so, how?",
    "Who provides flood insurance?",
    "Can you get flood insurance outside high-risk areas?",
    "How much in losses does fraud account for in property & casualty insurance?",
    "Do pay-as-you-drive insurance policies have an impact on greenhouse gas emissions? How much?",
    "What was the most costly earthquake in US history for insurers?",
    "Does it matter who is at fault to be compensated when injured on the job?"
]

## Set up Evaluation

In [6]:
import os
import numpy as np
from trulens_eval import Tru, Feedback, TruLlama, OpenAI as fOpenAI

from trulens_eval.feedback import Groundedness

openai = fOpenAI()

qa_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

qs_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
    .aggregate(np.mean)
)

# embedding distance
from langchain.embeddings.openai import OpenAIEmbeddings
from trulens_eval.feedback import Embeddings


#TODO fix embeddings
# model_name = 'text-embedding-ada-002'

# embed_model = OpenAIEmbeddings(
#     model=model_name,
#     openai_api_key=os.environ["OPENAI_API_KEY"]
# )

# embed = Embeddings(embed_model=embed_model)
# f_embed_dist = (
#     Feedback(embed.cosine_distance)
#     .on_input()
#     .on(TruLlama.select_source_nodes().node.text)
# )
#honest_feedbacks = [qa_relevance, qs_relevance, f_embed_dist, f_groundedness]


from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=openai)

f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(TruLlama.select_source_nodes().node.text.collect())
        .on_output()
        .aggregate(grounded.grounded_statements_aggregator)
)


honest_feedbacks = [qa_relevance, qs_relevance, f_groundedness]

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


 simple RAG often struggles with retrieving not enough information from the insurance manual to properly answer the question. The information needed may be just outside the chunk that is identified and retrieved by our app. Try sentence window retrieval to retrieve a wider chunk.

In [10]:
from llama_index.core.node_parser import SimpleFileNodeParser
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.indices.loading import load_index_from_storage
from llama_index.core.schema import Document
from llama_index.core.node_parser  import SentenceWindowNodeParser
from llama_index.core.indices.service_context import ServiceContext
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate 

import os

# initialize llm
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.5)

# knowledge store
document = Document(text="\n\n".join([doc.text for doc in documents]))

# set system prompt

system_prompt = PromptTemplate("We have provided context information below that you may use. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Please answer the question: {query_str}\n")

def build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3, 
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm, # fill llm
        embed_model=embed_model, # embed model
        node_parser=node_parser, # node parser
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            [document], service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index

sentence_index = build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
)

def get_sentence_window_query_engine(
    sentence_index,
    system_prompt,
    similarity_top_k=6, # top k
    rerank_top_n=2,
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank], text_qa_template = system_prompt
    )
    return sentence_window_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index, system_prompt=system_prompt)

tru_recorder_rag_sentencewindow = TruLlama(
        sentence_window_engine,
        app_id='2) Sentence Window RAG - Honest Eval',
        feedbacks=honest_feedbacks
    )

C:\Users\28263\AppData\Local\Temp\ipykernel_19424\1861917252.py:38: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

c:\Users\28263\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\28263\AppData\Local\llama_index\models\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [11]:
# Run evaluation on 10 sample questions
with tru_recorder_rag_sentencewindow as recording:
    for question in honest_evals:
        response = sentence_window_engine.query(question)

In [12]:
tru.get_leaderboard(app_ids=["1) Basic RAG - Honest Eval", "2) Sentence Window RAG - Honest Eval"])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
2) Sentence Window RAG - Honest Eval,0.133333,0.77,0.0,7.1,0.000732
